In [2]:
import pandas as pd
from pymilvus import connections, MilvusClient, DataType, Collection
from embedders import get_embedder
from database.database import Database

# connections.connect("default", host="localhost", port="19530")
client = MilvusClient(alias="default")
print(f"Client: {client}")

Client: <pymilvus.milvus_client.milvus_client.MilvusClient object at 0x121503710>


In [ ]:
# Instantiate pg database client
db = Database()
db.test_connection()

In [3]:
# Create the collection if it doesn't exist

COLLECTION_NAME = "contributions"
if not client.has_collection(collection_name=COLLECTION_NAME):
    print(f"Collection '{COLLECTION_NAME}' does not exist. Creating it.")
    schema = client.create_schema(
        auto_id=True,
        enable_dynamic_field=True
    )
    schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
    schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=2048)
    schema.add_field(field_name="doi", datatype=DataType.VARCHAR, max_length=64)
    schema.add_field(field_name="pubdate", datatype=DataType.INT64)  # Milvus has no date type
    schema.add_field(field_name="astrollama", datatype=DataType.FLOAT_VECTOR, dim=4096)

    client.create_collection(collection_name=COLLECTION_NAME, schema=schema)
print(f"Collections on client: {client.list_collections()}")
collection = Collection(COLLECTION_NAME)

Collections on client: ['chunks', 'basic_collection', 'contributions']


In [ ]:
# Instantiate astrollama embedder
embedder = get_embedder("UniverseTBD/astrollama", device="mps", normalize=False)
print(f"Embedder: {embedder}")

In [9]:
print([field.name for field in collection.schema.fields])

['id', 'text', 'doi', 'pubdate', 'astrollama']


In [ ]:
def rows_to_df(rows: list[tuple]) -> pd.DataFrame:
    """
    Expects a list of tuples (text, doi, pubdate)
    Embeds the text using the embedder to create a new column 'astrollama'
    Converts the pubdate in datetime.date format to an integer YYYYMMDD
    """
    df = pd.DataFrame(rows, columns=["text", "doi", "pubdate"])
    df["astrollama"] = embedder(df["text"]).tolist()  # Convert numpy array to list of lists
    df["pubdate"] = df["pubdate"].apply(lambda x: int(x.strftime("%Y%m%d")))  # Convert date to int YYYYMMDD
    return df
    

In [ ]:
# Iterate over DB contributions table and fetch data
BATCH_SIZE = 16
OFFSET = collection.num_entities
with db.conn.cursor() as cursor:
    cursor.execute("SELECT text, doi, pubdate FROM contributions OFFSET %s", (OFFSET,))
    while True:
        rows = cursor.fetchmany(BATCH_SIZE)
        if not rows:
            break
        df = rows_to_df(rows)

        # Insert into collection
        result = collection.insert(df)
        print(result)



In [ ]:
collection.

In [11]:
from tqdm import tqdm
# Create new collection with both vector fields
NEW_COLLECTION_NAME = "contributions_with_qwen"

if not client.has_collection(collection_name=NEW_COLLECTION_NAME):
    print(f"Creating new collection '{NEW_COLLECTION_NAME}' with both vector fields.")
    schema = client.create_schema(auto_id=True, enable_dynamic_field=True)
    schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
    schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=2048)
    schema.add_field(field_name="doi", datatype=DataType.VARCHAR, max_length=64)
    schema.add_field(field_name="pubdate", datatype=DataType.INT64)
    schema.add_field(field_name="astrollama", datatype=DataType.FLOAT_VECTOR, dim=4096)
    schema.add_field(field_name="qwen8b", datatype=DataType.FLOAT_VECTOR, dim=4096)  # New field

    client.create_collection(collection_name=NEW_COLLECTION_NAME, schema=schema)

new_collection = Collection(NEW_COLLECTION_NAME)
old_collection = Collection("contributions")

# Load embedder
qwen_embedder = get_embedder("Qwen/Qwen3-Embedding-8B", device="mps", normalize=True)
print(f"Qwen Embedder: {qwen_embedder}")

# Migrate data with new embeddings
BATCH_SIZE = 8
total_entities = old_collection.num_entities
processed = 0

print(f"Migrating {total_entities} entities from 'contributions' to '{NEW_COLLECTION_NAME}'")

with tqdm(total=total_entities, desc="Migrating with Qwen embeddings") as pbar:
    for offset in range(0, total_entities, BATCH_SIZE):
        # Query batch from old collection
        entities = old_collection.query(
            expr="", output_fields=["text", "doi", "pubdate", "astrollama"], limit=BATCH_SIZE, offset=offset
        )

        if not entities:
            break

        # Extract data for new collection
        texts = [entity["text"] for entity in entities]
        dois = [entity["doi"] for entity in entities]
        pubdates = [entity["pubdate"] for entity in entities]
        astrollama_embeddings = [entity["astrollama"] for entity in entities]

        # Generate new Qwen embeddings
        qwen_embeddings = qwen_embedder(texts)

        # Prepare data for insertion
        insert_data = {
            "text": texts,
            "doi": dois,
            "pubdate": pubdates,
            "astrollama": astrollama_embeddings,
            "qwen8b": qwen_embeddings.tolist(),
        }

        # Insert into new collection
        result = new_collection.insert(insert_data)
        processed += len(entities)
        pbar.update(len(entities))

print(f"Migration complete! {processed} entities migrated.")

# Create indexes on the new collection
new_collection.create_index(
    field_name="astrollama",
    index_params={
        "index_type": "FLAT",
        "metric_type": "IP",
    },
)

new_collection.create_index(
    field_name="qwen8b",
    index_params={
        "index_type": "FLAT",
        "metric_type": "COSINE",
    },
)

# Optional: Drop old collection and rename new one
client.drop_collection("contributions")
client.rename_collection(old_name=NEW_COLLECTION_NAME, new_name="contributions")

2025-08-12 14:51:27,411 [ERROR][handler]: RPC error: [create_collection], <MilvusException: (code=0, message=multiple vector fields is not supported, fields name: astrollama, qwen8b)>, <Time:{'RPC start': '2025-08-12 14:51:27.409351', 'RPC error': '2025-08-12 14:51:27.411104'}> (decorators.py:140)
2025-08-12 14:51:27,411 [ERROR][_create_collection_with_schema]: Failed to create collection: contributions_with_qwen (milvus_client.py:912)


Creating new collection 'contributions_with_qwen' with both vector fields.


MilvusException: <MilvusException: (code=0, message=multiple vector fields is not supported, fields name: astrollama, qwen8b)>

In [ ]:
collection.create_index(
    field_name="astrollama",
    index_params={
        "index_type": "FLAT",
        "metric_type": "L2",
    },
)

index_params = client.prepare_index_params()
index_params.add_index(
    field_name="pubdate",
    index_type="STL_SORT",
    index_name="contributions_pubdate_index",
)

client.create_index(
    collection_name=collection.name,
    index_params=index_params
)

In [ ]:
# Create the collection if it doesn't exist

COLLECTION_NAME = "chunks"
if not client.has_collection(collection_name=COLLECTION_NAME):
    print(f"Collection '{COLLECTION_NAME}' does not exist. Creating it.")
    schema = client.create_schema(auto_id=True, enable_dynamic_field=True)
    schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
    schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=2048)
    schema.add_field(field_name="doi", datatype=DataType.VARCHAR, max_length=64)
    schema.add_field(field_name="pubdate", datatype=DataType.INT64)  # Milvus has no date type
    schema.add_field(field_name="astrollama", datatype=DataType.FLOAT_VECTOR, dim=4096)

    client.create_collection(collection_name=COLLECTION_NAME, schema=schema)
print(f"Collections on client: {client.list_collections()}")
collection = Collection(COLLECTION_NAME)
print(f"{collection.num_entities} entities in collection {COLLECTION_NAME}")

In [ ]:
# Iterate over DB contributions table and fetch data
with db.conn.cursor() as cursor:
    cursor.execute("SELECT COUNT(*) FROM chunks")
    total_chunks = cursor.fetchone()[0]

processed = 0
print(f"Total chunks to process: {total_chunks}")


In [ ]:
# db.conn.rollback()

In [ ]:
BATCH_SIZE = 8
with db.conn.cursor() as cursor:
    cursor.execute("SELECT text, doi, pubdate FROM chunks")
    while True:
        rows = cursor.fetchmany(BATCH_SIZE)
        # print(f"Fetched {len(rows)} rows.")
        if not rows:
            break
        df = rows_to_df(rows)
        # print(f"Converted rows to DataFrame with {len(df)} entries.")

        # Insert into collection
        result = collection.insert(df)
        processed += len(rows)
        print(f"\rProcessed {processed}/{total_chunks} total chunks.", end="", flush=True)